In [80]:
from sklearn.datasets import load_boston
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score, ShuffleSplit
import numpy as np
import pandas as pd

In [81]:
dataset = pd.read_csv(r'C:\Users\thinkpad\Documents\GitHub\PHBS_MLF_2018\data\5year.arff.csv')
target='class'#二分类的名称
x_columns = [x for x in dataset.columns if x not in [target]]
X = dataset[x_columns]
y = dataset['class']
names = X.columns
Xarray = X.values
yarray = y.values
ranks = {}

#  Univariate feature selection

### Pearson Correlation

In [82]:
f, pval  = f_classif(Xarray, yarray)
ranks = sorted(zip(f,names), reverse=False)
Ranks = pd.DataFrame(ranks)
print(Ranks)
#ranks["Corr."] = rank_to_dict(f, names)
#print(ranks["Corr."])

            0       1
0    0.000240  Attr60
1    0.003201  Attr30
2    0.009139  Attr40
3    0.012636  Attr19
4    0.012922  Attr31
5    0.013476  Attr23
6    0.013824  Attr36
7    0.016130  Attr17
8    0.016293   Attr8
9    0.020892  Attr34
10   0.023183   Attr9
11   0.026314  Attr32
12   0.026893  Attr61
13   0.029714  Attr52
14   0.035845  Attr13
15   0.039708  Attr45
16   0.042001  Attr47
17   0.042305  Attr33
18   0.051074  Attr59
19   0.057061   Attr4
20   0.058484  Attr50
21   0.074983  Attr57
22   0.075364  Attr63
23   0.076873  Attr56
24   0.095519  Attr37
25   0.109355  Attr21
26   0.115052  Attr46
27   0.116955  Attr44
28   0.129571  Attr39
29   0.131145  Attr43
..        ...     ...
34   0.154845  Attr14
35   0.157127  Attr20
36   0.158320  Attr22
37   0.163233  Attr12
38   0.163914  Attr48
39   0.169245  Attr18
40   0.170397  Attr49
41   0.272182  Attr26
42   0.310165  Attr35
43   0.313438  Attr27
44   0.316875  Attr16
45   0.386690  Attr28
46   0.498105   Attr5
47   0.725

### Random Forest Regression Based Ranking

# Logistic model and regularization

### L1 regularization

In [72]:
lr = LogisticRegression(C=1.0, penalty="l1", random_state=0)
lr.fit(Xarray,yarray)
lrcoef = lr.coef_[-1]
ranks = sorted(zip(np.abs(lrcoef),names), reverse=True)
Ranks = pd.DataFrame(ranks)
print(Ranks)

KeyboardInterrupt: 

### L2 regularization

In [73]:
lr = LogisticRegression(C=1.0, penalty="l2", random_state=0)
lr.fit(Xarray,yarray)
lrcoef = lr.coef_[-1]
ranks = sorted(zip(np.abs(lrcoef),names), reverse=True)
Ranks = pd.DataFrame(ranks)
print(Ranks)

               0       1
0   4.730422e-03  Attr43
1   2.725520e-03  Attr44
2   2.517781e-03  Attr47
3   2.019976e-03  Attr62
4   2.004903e-03  Attr20
5   1.282586e-03  Attr60
6   9.715565e-04  Attr64
7   7.260473e-04  Attr37
8   7.236187e-04  Attr32
9   6.301370e-04  Attr61
10  4.118972e-04  Attr63
11  3.614573e-04  Attr33
12  2.549255e-04  Attr21
13  2.098171e-04  Attr29
14  1.933213e-04  Attr27
15  1.905867e-04  Attr54
16  1.815441e-04  Attr34
17  1.775225e-04  Attr53
18  1.762991e-04  Attr17
19  1.211366e-04   Attr8
20  1.196958e-04  Attr36
21  1.088147e-04   Attr4
22  1.062218e-04  Attr28
23  1.001697e-04   Attr9
24  8.837912e-05  Attr50
25  7.516105e-05  Attr46
26  7.028807e-05  Attr45
27  4.501297e-05  Attr58
28  4.374148e-05  Attr16
29  4.221692e-05  Attr26
..           ...     ...
34  2.930446e-05  Attr25
35  2.558839e-05  Attr40
36  1.814923e-05  Attr57
37  1.777898e-05   Attr2
38  1.645405e-05   Attr3
39  1.529624e-05  Attr55
40  1.486818e-05  Attr59
41  1.263585e-05  Attr24


# Random Forest（multivariable）

In [ ]:
rf = RandomForestRegressor()
rf.fit(Xarray,yarray)
ranks = sorted(zip((rf.feature_importances_ , names), reverse=True)
Ranks = pd.DataFrame(ranks)
print(Ranks)

# Recursive Feature Elimination 

In [ ]:
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(Xarray,yarray)
ranks = sorted(zip((rfe.ranking_ , names), reverse=False)
Ranks = pd.DataFrame(ranks)
print(Ranks)